In [ ]:
## LIBRERIAS
from google.colab import drive, files
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras import Model, Sequential
from keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, Rescaling
import random
import os
random.seed(1234)

In [ ]:
## Cargamos Drive e importamos las imágenes de train y test
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/Colab/Sentinel2LULC_354.zip" -d "/content"
!unzip "/content/drive/MyDrive/Colab/Sentinel2LULC_Test.zip" -d "/content"

In [ ]:
## Renombramos las carpetas para que tf las lea en orden
dirnames = sorted(os.listdir("Sentinel2LULC_354"))
for dir in dirnames:
  newname = ""

  if (len(dir.split('_')[0]) == 1):
    newname = '0'+dir.split('_')[0]
  else:
    newname = dir.split('_')[0]

  os.rename("Sentinel2LULC_354/"+dir,"Sentinel2LULC_354/"+newname)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_image_generator = ImageDataGenerator(horizontal_flip = True,
                                           vertical_flip = True,
                                           validation_split = 0.2)


training_generator = train_image_generator.flow_from_directory(
  "Sentinel2LULC_354",
  subset="training",
  seed=1234,
  target_size=(224, 224),
  batch_size = batch_size,
  class_mode='sparse')


validation = tf.keras.utils.image_dataset_from_directory(
  "Sentinel2LULC_354",
  validation_split=0.2,
  subset="validation",
  seed=1234,
  image_size=(224, 224),
  batch_size = 32)


test = tf.keras.utils.image_dataset_from_directory(
  "Sentinel2LULC_Test",
  shuffle=False,
  labels=None,
  image_size=(224, 224),
  batch_size = 32)

total_train = 8236
total_val = 2030

Found 8236 images belonging to 29 classes.
Found 10266 files belonging to 29 classes.
Using 2053 files for validation.
Found 1618 files belonging to 1 classes.


In [ ]:

## Vamos a entrenar usando la red InceptionV3
base_model = tf.keras.applications.VGG19(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling = "avg")

for layer in base_model.layers:
  if any(map(lambda v: v in layer.name, ["block1","block2"])):
    layer.trainable = False
  else:
    print(layer.name)
    



input_3
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
global_average_pooling2d_2


In [ ]:
normalization = Rescaling(1./255)

model = Sequential([
  normalization,
  base_model,
  Dense(1024, activation="relu"),
  Dropout(0.2),
  Dense(256, activation="relu"),
  Dropout(0.2),
  Dense(29, activation="softmax")
])

opt = SGD(learning_rate=1e-3)
      
model.compile(loss= SparseCategoricalCrossentropy(from_logits = False), optimizer=opt, 
                  metrics=['accuracy'])

# a partir de TF 2.2, fit ya funciona con generadores, no hace falta usar fit_generator
callback = EarlyStopping(patience=5,restore_best_weights=True)
history = model.fit(training_generator,
                        epochs=50,
                        steps_per_epoch = total_train // batch_size, # si queremos usar más imágenes, aumentar este parametro y validation_steps
                        validation_data = validation,
                        validation_steps = total_val // batch_size,
                        callbacks=callback)

In [ ]:
## Predecimos las clases
y_pred = model.predict(test)
classes = np.argmax(y_pred, axis=1)
classes = np.array(classes)
classes = classes + 1
print(classes)

[10 10 10 ...  9  9 20]


In [ ]:
## Creamos el csv con los resultados
!rm Sentinel2LULC_Test/.nomedia
d = {'id.jpg': sorted(os.listdir("Sentinel2LULC_Test")), 'label': classes}
df = pd.DataFrame(data=d)
df.to_csv('/content/drive/MyDrive/Colab/resultados.csv', index=False)